In [177]:
%matplotlib ipympl
%load_ext line_profiler
import gym
import gym_watten
from gym_watten.envs.watten_env import Color, Value
import numpy as np
from time import sleep
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import pydot_ng as pydot
from io import BytesIO
import pickle

In [2]:
env = gym.make("Watten-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_watten.envs.watten_env.WattenEnv'>' has deprecated methods. Compatibility code invoked.


In [217]:
EPISODES = 75
MCTS_SIMS = 100
MEMORY_SIZE = 300000
SAMPLES_PER_STATE = 1

BATCH_SIZE = 256
TRAINING_LOOPS = 100
EXPLORATION = 1

EPSILON = 0.2
ALPHA = 0.8